# TensorFlow GPU avec Azure ML

<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>

In [1]:
import sys
sys.version

'3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) \n[GCC 7.3.0]'

In [2]:
import utils

In [3]:
import datetime
maintenant = datetime.datetime.now()
print("Date du run = ", maintenant)

Date du run =  2020-05-06 09:48:57.014619


In [4]:
import azureml.core
print("Version Azure ML =", azureml.core.VERSION)

Version Azure ML = 1.4.0


## 1. Workspace

In [5]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

## 2. AML Compute GPU

Les tailles NC, NCv2 et NCv3 sont optimisées pour les algorithmes et les applications nécessitant beaucoup de ressources réseau et de calculs. 

En voici quelques exemples : les applications et les simulations CUDA et OpenCL, l’intelligence artificielle et l’apprentissage profond. Équipée du GPU Tesla V100 de NVIDIA, la série NCv3 est axée sur les charges de travail informatiques à hautes performances. 

- La série NC utilise le processeur Intel Xeon E5-2690 v3 2.60GHz v3 (Haswell) et les machines virtuelles de la série NCv2 et NCv3 sont dotées du processeur Intel Xeon E5-2690 v4 (Broadwell).

- ND et NDv2La série ND est destinée à l’exécution de scénarios d’apprentissage et d’inférence pour le Deep Learning. Elle utilise le GPU NVIDIA Tesla P40 et le processeur Intel Xeon E5-2690 v4 (Broadwell). La série NDv2 utilise le processeur Intel Xeon Platinum 8168 (Skylake).

- Les tailles NV et NVv3 sont optimisées et conçues pour la visualisation à distance, la diffusion en continu, les jeux, l’encodage et les scénarios de VDI utilisant des infrastructures comme OpenGL ou DirectX. Ces machines virtuelles reposent sur le GPU Tesla M60 de NVIDIA.

- Les tailles NVv4 sont optimisées et conçues pour l’infrastructure VDI et la visualisation à distance. Avec des GPU partitionnés, NVv4 offre la taille adaptée aux charges de travail nécessitant des ressources GPU plus petites. Ces machines virtuelles sont associées au GPU AMD Radeon Instinct MI25.

https://docs.microsoft.com/fr-fr/azure/virtual-machines/windows/sizes-gpu

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

AmlCompute.supported_vmsizes(workspace = ws)

[{'name': 'Standard_D1_v2',
  'vCPUs': 1,
  'gpus': 0,
  'memoryGB': 3.5,
  'maxResourceVolumeMB': 51200},
 {'name': 'Standard_D2_v2',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 7.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D3_v2',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D4_v2',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D11_v2',
  'vCPUs': 2,
  'gpus': 0,
  'memoryGB': 14.0,
  'maxResourceVolumeMB': 102400},
 {'name': 'Standard_D12_v2',
  'vCPUs': 4,
  'gpus': 0,
  'memoryGB': 28.0,
  'maxResourceVolumeMB': 204800},
 {'name': 'Standard_D13_v2',
  'vCPUs': 8,
  'gpus': 0,
  'memoryGB': 56.0,
  'maxResourceVolumeMB': 409600},
 {'name': 'Standard_D14_v2',
  'vCPUs': 16,
  'gpus': 0,
  'memoryGB': 112.0,
  'maxResourceVolumeMB': 819200},
 {'name': 'Standard_DS1_v2',
  'vCPUs': 1,
  'gpus': 0,
  'memoryGB': 3.5,
  'maxResourceVolumeMB': 7168},
 {'name': 'Standar

In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpuclusterNC6"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           min_nodes=1,
                                                           max_nodes=8
                                                           )

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target...
Creating
Succeeded..................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-05-06T09:50:47.946000+00:00', 'errors': None, 'creationTime': '2020-05-06T09:49:01.748882+00:00', 'modifiedTime': '2020-05-06T09:49:20.614926+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 8, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [8]:
liste = ws.compute_targets
for liste in liste:
    print("Ressources Compute du workspace :", liste)

Ressources Compute du workspace : drift-aks
Ressources Compute du workspace : instance-aks
Ressources Compute du workspace : instance
Ressources Compute du workspace : AutoML
Ressources Compute du workspace : cpu-cluster
Ressources Compute du workspace : cpupipelines
Ressources Compute du workspace : clustergpuNC6
Ressources Compute du workspace : gpuclusterNC6


In [9]:
compute_target.get_status().serialize()

{'currentNodeCount': 1,
 'targetNodeCount': 1,
 'nodeStateCounts': {'preparingNodeCount': 1,
  'runningNodeCount': 0,
  'idleNodeCount': 0,
  'unusableNodeCount': 0,
  'leavingNodeCount': 0,
  'preemptedNodeCount': 0},
 'allocationState': 'Steady',
 'allocationStateTransitionTime': '2020-05-06T09:50:47.946000+00:00',
 'errors': None,
 'creationTime': '2020-05-06T09:49:01.748882+00:00',
 'modifiedTime': '2020-05-06T09:49:20.614926+00:00',
 'provisioningState': 'Succeeded',
 'provisioningStateTransitionTime': None,
 'scaleSettings': {'minNodeCount': 1,
  'maxNodeCount': 8,
  'nodeIdleTimeBeforeScaleDown': 'PT120S'},
 'vmPriority': 'Dedicated',
 'vmSize': 'STANDARD_NC6'}

In [10]:
# Statut du compute server
compute_target.list_nodes()

[{'nodeId': 'tvmps_c69fa2f20abdf20a91d71f7f498bd71650cc2b0a081c65179aa76fc79c487bab_d',
  'port': 50000,
  'publicIpAddress': '20.50.146.19',
  'privateIpAddress': '10.0.0.4',
  'nodeState': 'preparing'}]

## 3. Données

In [11]:
from azureml.core.dataset import Dataset
web_paths = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

In [12]:
dataset = dataset.register(workspace = ws,
                           name = 'mnist dataset',
                           description='training and test dataset',
                           create_new_version=True)

In [13]:
dataset.to_path()

['/http/yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
 '/http/yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz']

## 4. Création projet et expérimentation

In [14]:
import os
script_folder = './tf-resume-training'
os.makedirs(script_folder, exist_ok=True)

In [15]:
from azureml.core import Experiment

experiment_name = 'Exemple11-TensorFlow'
experiment = Experiment(ws, name=experiment_name)

## 5. Création et exécution estimator TensorFlow

### Visualisation du code python :

In [16]:
with open(os.path.join(script_folder, './tf_mnist_with_checkpoint.py'), 'r') as f:
    print(f.read())

# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import numpy as np
import utils
import argparse
import os
import re
import tensorflow as tf
import glob

from azureml.core import Run
from utils import load_data

print("TensorFlow version:", tf.__version__)

parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')

parser.add_argument('--resume-from', type=str, default=None,
                    help='location of the model or checkpoint files from where to resume the training')
args = parser.parse_args()


previous_model_location = args.resume_from
# You can also use environment variable to get the model/checkpoint files location
# previous_model_location = os.path.expandvars(os.getenv("AZUREML_DATAREFERENCE_MODEL_LOCATION", None))

data_folder = args.data_folder
print('Data folder:', data_folder)

# load train and test set into numpy arrays
# note we scale the pi

In [17]:
from azureml.train.dnn import TensorFlow

script_params={
    '--data-folder': dataset.as_named_input('mnist').as_mount()
}

estimator= TensorFlow(source_directory=script_folder,
                      compute_target=compute_target,
                      script_params=script_params,
                      entry_script='tf_mnist_with_checkpoint.py',
                      use_gpu=True,
                      pip_packages=['azureml-dataprep[pandas,fuse]'])

WARNING - framework_version is not specified, defaulting to version 1.13.
WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['azureml-dataprep']. We cannot guarantee image build will succeed.


### Exécution du Run

In [18]:
tags = {"Type": "test" , "Langage" : "Python" , "Framework"  : "TensorFlow"}

In [19]:
run = experiment.submit(estimator, tags=tags)
print(run)

Run(Experiment: Exemple11-TensorFlow,
Id: Exemple11-TensorFlow_1588758679_92122b24,
Type: azureml.scriptrun,
Status: Starting)


### Widget pour suivi de l'exécution du run

In [20]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [24]:
run.get_details()

{'runId': 'Exemple11-TensorFlow_1588758679_92122b24',
 'target': 'gpuclusterNC6',
 'status': 'Completed',
 'startTimeUtc': '2020-05-06T09:52:04.868434Z',
 'endTimeUtc': '2020-05-06T09:55:23.625093Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': 'dd63f8ac-5a18-49b0-9a1d-cecb24d36ddd',
  'azureml.git.repository_uri': 'https://github.com/retkowsky/WorkshopAML2020',
  'mlflow.source.git.repoURL': 'https://github.com/retkowsky/WorkshopAML2020',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'eb05ad565a41b5121d26c6fda4b1c6398a9243d7',
  'mlflow.source.git.commit': 'eb05ad565a41b5121d26c6fda4b1c6398a9243d7',
  'azureml.git.dirty': 'True',
  'AzureML.DerivedImageName': 'azureml/azureml_1b21012d99b68bea171596e10bcaefa4',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '15fab2af-43d1-4754-b247-a4961f9a327

### Capture d'écran des résultats du run :

<img src="https://github.com/retkowsky/images/blob/master/GPUTF.jpg?raw=true">

### Liste des métriques du run

In [25]:
run.get_metrics()

{'training_acc': [0.9399999976158142,
  0.9599999785423279,
  0.8999999761581421,
  0.8600000143051147,
  0.9399999976158142,
  0.9800000190734863,
  0.9599999785423279,
  0.9399999976158142,
  0.9399999976158142,
  0.9599999785423279,
  0.9599999785423279],
 'validation_acc': [0.8964999914169312,
  0.9172999858856201,
  0.9247999787330627,
  0.9337999820709229,
  0.9369999766349792,
  0.9401999711990356,
  0.9467999935150146,
  0.9490000009536743,
  0.9510999917984009,
  0.9534000158309937,
  0.9562000036239624],
 'final_acc': 0.9562000036239624}

### Visualisation des métriques dans l'expérimentation depuis Azure ML Studio

In [26]:
experiment

Name,Workspace,Report Page,Docs Page
Exemple11-TensorFlow,workshopAML2020,Link to Azure Machine Learning studio,Link to Documentation


In [27]:
# Statut du compute server
compute_target.list_nodes()

[{'nodeId': 'tvmps_7741159579982403935e773490d11f29de52c00b698a36319ade58520cfb4f89_d',
  'port': 50002,
  'publicIpAddress': '20.50.146.19',
  'privateIpAddress': '10.0.0.6',
  'nodeState': 'running',
  'runId': 'exemple13-tfhorovod_1588758773_de8dce9e'},
 {'nodeId': 'tvmps_822eabb6ca44f083ff0056d1d44961af0bc7df36033cd11d4a92a279fcd9c3bd_d',
  'port': 50001,
  'publicIpAddress': '20.50.146.19',
  'privateIpAddress': '10.0.0.5',
  'nodeState': 'running',
  'runId': 'exemple13-tfhorovod_1588758773_de8dce9e'},
 {'nodeId': 'tvmps_c69fa2f20abdf20a91d71f7f498bd71650cc2b0a081c65179aa76fc79c487bab_d',
  'port': 50000,
  'publicIpAddress': '20.50.146.19',
  'privateIpAddress': '10.0.0.4',
  'nodeState': 'idle'}]

### Suppression compute instance

In [28]:
liste = ws.compute_targets
for liste in liste:
    print("Ressources Compute du workspace :", liste)

Ressources Compute du workspace : drift-aks
Ressources Compute du workspace : instance-aks
Ressources Compute du workspace : instance
Ressources Compute du workspace : AutoML
Ressources Compute du workspace : cpu-cluster
Ressources Compute du workspace : cpupipelines
Ressources Compute du workspace : clustergpuNC6
Ressources Compute du workspace : gpuclusterNC6
Ressources Compute du workspace : cpu-standardd4


In [29]:
# Suppression du cluster
#compute_target.delete()

In [30]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, "(" , ct.type, ") :", ct.provisioning_state)

drift-aks ( AKS ) : Succeeded
instance-aks ( AKS ) : Succeeded
instance ( ComputeInstance ) : Succeeded
AutoML ( AmlCompute ) : Succeeded
cpu-cluster ( AmlCompute ) : Succeeded
cpupipelines ( AmlCompute ) : Succeeded
clustergpuNC6 ( AmlCompute ) : Succeeded
gpuclusterNC6 ( AmlCompute ) : Succeeded
cpu-standardd4 ( AmlCompute ) : Succeeded


<img src="https://github.com/retkowsky/images/blob/master/Powered-by-MS-Azure-logo-v2.png?raw=true" height="300" width="300">